In [2]:
from keras.preprocessing import image
from keras.layers import Dense, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from keras.models import Sequential, Model
from keras import layers, callbacks, utils, regularizers
from keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.utils import image_dataset_from_directory
from keras.callbacks import LearningRateScheduler
import tensorflow as tf
from tensorflow import keras

import os, pathlib, random
import numpy as np
import itertools


keras.utils.set_random_seed(42)

In [46]:
from utils_fn import generate_dataset, plot_confusion_matrix, plot_history, generate_classification_report

In [42]:
!pip list


Package                      Version
---------------------------- ---------------
absl-py                      2.1.0
asttokens                    2.4.1
astunparse                   1.6.3
cachetools                   5.4.0
certifi                      2024.7.4
charset-normalizer           3.3.2
colorama                     0.4.6
comm                         0.2.2
debugpy                      1.8.2
decorator                    5.1.1
exceptiongroup               1.2.2
executing                    2.0.1
flatbuffers                  24.3.25
gast                         0.4.0
google-auth                  2.32.0
google-auth-oauthlib         0.4.6
google-pasta                 0.2.0
grpcio                       1.65.1
h5py                         3.11.0
idna                         3.7
importlib_metadata           8.0.0
ipykernel                    6.29.5
ipython                      8.26.0
jedi                         0.19.1
jupyter_client               8.6.2
jupyter_core                 5.7.2

In [27]:
DATASET_DIR = pathlib.Path("Ro_Sign_language_Dataset")
MODEL_SAVE_DIR = pathlib.Path("Saves/V2")
SAVE_NAME = f'Bare model'

BATCH_SIZE = 64
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
EPOCHS = 50
UNITS = 256

In [28]:
train_dataset, validation_dataset, test_dataset = generate_dataset(base_dir=DATASET_DIR, image_size=(IMAGE_HEIGHT, IMAGE_WIDTH), batch_size=BATCH_SIZE)

Found 15600 files belonging to 26 classes.
Found 15600 files belonging to 26 classes.
Found 5200 files belonging to 26 classes.


In [29]:
rescale = layers.Rescaling(1./255)

inputs = keras.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3))
x = rescale(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
x = layers.Dense(UNITS, activation='relu')(x)
outputs = layers.Dense(26, activation="softmax")(x)
my_model = keras.Model(inputs=inputs, outputs=outputs)

my_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 128, 128, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                         

In [30]:
my_model.compile(loss="categorical_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])

In [32]:
callbacks = [keras.callbacks.ModelCheckpoint(filepath = 'b.model.keras',
                                             save_best_only=True,
                                             monitor="val_loss")]

history = my_model.fit(train_dataset,
                       epochs=10,
                       validation_data=validation_dataset,
                       callbacks=callbacks)

np.save('hh.history.npy', history.history)

Epoch 1/10
244/244 [==============================] - 74s 277ms/step - loss: 1.7758 - accuracy: 0.4665 - val_loss: 0.2987 - val_accuracy: 0.9095
Epoch 2/10
244/244 [==============================] - 15s 59ms/step - loss: 0.2124 - accuracy: 0.9319 - val_loss: 0.1139 - val_accuracy: 0.9685
Epoch 3/10
244/244 [==============================] - 23s 96ms/step - loss: 0.0759 - accuracy: 0.9778 - val_loss: 0.0426 - val_accuracy: 0.9881
Epoch 4/10
244/244 [==============================] - 14s 55ms/step - loss: 0.0370 - accuracy: 0.9888 - val_loss: 0.0831 - val_accuracy: 0.9737
Epoch 5/10
244/244 [==============================] - 14s 55ms/step - loss: 0.0327 - accuracy: 0.9900 - val_loss: 0.0147 - val_accuracy: 0.9957
Epoch 6/10
244/244 [==============================] - 14s 56ms/step - loss: 0.0275 - accuracy: 0.9915 - val_loss: 0.0190 - val_accuracy: 0.9946
Epoch 7/10
244/244 [==============================] - 14s 57ms/step - loss: 0.0176 - accuracy: 0.9956 - val_loss: 0.0168 - val_accuracy

In [33]:
!pip freeze > requirements.txt

In [35]:
test_loss, test_acc = my_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

82/82 [==============================] - 13s 153ms/step - loss: 1.1972 - accuracy: 0.8256
Test accuracy: 0.826
